In [ ]:
# libary imports 
import os
import torch
from torch import nn
import torchvision
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
from torchvision import transforms
import torchvision.datasets as datasets

from sklearn import metrics
from sklearn import decomposition
from sklearn import manifold
from tqdm.notebook import trange, tqdm

import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data

import matplotlib.pyplot as plt
import numpy as np
import copy
import random
import time


from functools import reduce
import operator

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
from torch.utils.data import Subset


# Prepare CIFAR-10 dataset
trainset = CIFAR10(os.getcwd(), download=True, transform=transforms.Compose(
      [transforms.ToTensor(),
      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]))
trainloader = torch.utils.data.DataLoader(trainset, batch_size=8, shuffle=False, num_workers=2)

testset = CIFAR10(os.getcwd(), download=True, transform=transforms.Compose(
      [transforms.ToTensor(),
      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]))
testloader = torch.utils.data.DataLoader(testset, batch_size=8, shuffle=False, num_workers=2)



  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting /content/cifar-10-python.tar.gz to /content
Files already downloaded and verified


In [ ]:

class MLPClassifer(nn.Module):
      '''
        Multilayer Perceptron.
      '''
      def __init__(self ):
        super().__init__()
        input=trainset.data[1].shape
        output=len(torch.unique(torch.tensor(trainset.targets)))
        hidden_layer_sizes=(64,32)
        self.layers = nn.Sequential(
          nn.Flatten(),
          nn.Linear(reduce(operator.mul, input),  hidden_layer_sizes[0]),
          nn.ReLU(),
          nn.Linear(hidden_layer_sizes[0],  hidden_layer_sizes[1]),
          nn.ReLU(),
          nn.Linear(hidden_layer_sizes[1], output)
        )

      def forward(self, x):
        '''Forward pass'''
        return self.layers(x)

class Neural_Network(object):
    
    
    def __init__ (self,batch_size=2000,epochs=2,num_classes=0,verbose=True,running_loss=0,loss_function = nn.CrossEntropyLoss(),
         optimiser_choice='adam',classifier= MLPClassifer(), 
         learning_rate=1e-3,  betas=(0.9,0.99) , eps=1e-8, weight_decay=0, amsgrad=False,
      momentum=0.9, nesterov=True, dampening=0):
      
      self.batch_size=batch_size
      self.epochs=epochs
      #used for label count 
      self.num_classes=0
      self.verbose=True
        
      self.running_loss = running_loss
      self.loss_function = loss_function
       
      self.classifier= classifier
      self.optimiser_choice=optimiser_choice
      
      #needed for extraxting the probabilities 
      self.sm = torch.nn.Softmax()

      #some initial parameters for either standard adam ot sgd

      self.learning_rate=learning_rate 
      self.betas= betas
      self.eps= eps
      self.weight_decay=weight_decay
      self.amsgrad=amsgrad 
      self.momentum=momentum
      self.nesterov=nesterov
      self.dampening=dampening
      self.optimiser=None 
      self.set_optimiser(self.optimiser_choice)
       

    def get_optimiser(self):
      x=self.set_optimiser(self.optimiser_choice)
      print(x)


    def set_optimiser(self, optimiser_choice):
      #playing around with adam and sgd only at the moment
        print('We are outside this function')
        if  optimiser_choice.lower().strip()=='adam':
          self.optimiser=torch.optim.Adam(self.classifier.parameters(),lr=self.learning_rate, betas=self.betas, eps=self.eps, weight_decay=self.weight_decay, amsgrad=self.amsgrad)
          print(' Check whether the optimiser is initialised correctly')
        elif optimiser_choice.lower().strip()=='sgd':
          self.optimiser=torch.optim.SGD(self.classifier.parameters(),lr=self.learning_rate, weight_decay=self.weight_decay, momentum=0.9,nesterov=self.nesterov , dampening=self.dampening)
        return self.optimiser


    def get_params(self): #get parameters
       if (self.optimiser != None) :
         return  (self.props(),self.optimiser.state_dict(), self.classifier.state_dict)
       else: return self.classifier.state_dict() 
  
    def fit(self,trainloader):
        for epoch in range(self.epochs):  # loop over the dataset 2 times

          running_loss =self.running_loss
          for i, data in enumerate(trainloader, 0):
          # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
             
            # zero the parameter gradients
            self.optimiser.zero_grad()

            # forward + compute loss+ backward + optimize
            outputs = self.classifier(inputs)
            loss = self.loss_function(outputs, labels)
            loss.backward()
            self.optimiser.step()
            # print statistics
            running_loss += loss.item()
            if i % self.batch_size == 1999:    # print every 2000 mini-batches
                if self.verbose==True:
                  print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / self.batch_size:.3f}')
                running_loss = 0.0

    def predict(self,testloader): #Predict using the multi-layer perceptron classifier.
      prediction_list=[]
      # again no gradients needed
      with torch.no_grad():
          for data in testloader:
              images, labels = data
              outputs = self.classifier(images)
              estimation, predictions = torch.max(outputs, 1)
              prediction_list.append(predictions)
      return prediction_list #torch.stack(prediction_list)


    def predict_log_proba(self,loader):  #	Return the log of probability estimates.
        y_prob=self.predict_proba(loader)
        log_proba=np.log(y_prob)
        return log_proba
 

    def predict_proba(self,loader):	#Probability estimates.
      probabilities_list=[]
      # again no gradients needed
      with torch.no_grad():
          for data in loader:
              images, labels = data
              outputs = self.classifier(images)
              probabilities_list.append(self.sm(outputs) )
              estimation, predictions = torch.max(outputs, 1)
      return probabilities_list
      

    def score(self,loader): #Return the mean accuracy on the given test data and labels.
        prediction_list=[]
        if self.num_classes==0:
            classes = ('plane', 'car', 'bird', 'cat',
                    'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
            # prepare to count predictions for each class
            correct_pred = {classname: 0 for classname in classes}
            total_pred = {classname: 0 for classname in classes}
        #use the class number as key
        else: 
            classes=tuple(np.arange(self.num_classes))  
            correct_pred = {classname: 0 for classname in np.arange(self.num_classes)}
            total_pred= {classname: 0 for classname in np.arange(self.num_classes)}

        # again no gradients needed
        with torch.no_grad():
            for data in loader:
                images, labels = data
                outputs = self.classifier(images)
                estimation, predictions = torch.max(outputs, 1)
                prediction_list.append(predictions)
                # collect the correct predictions for each class
                for label, prediction in zip(labels, predictions):
                    if label == prediction:
                        correct_pred[classes[label]] += 1
                    total_pred[classes[label]] += 1
        accuracies={}
        # print accuracy for each class
        for classname, correct_count in correct_pred.items():
            accuracy = 100 * float(correct_count) / total_pred[classname]
            accuracies[classname]=accuracy
            if self.verbose:
              print(f'Accuracy for class: {classname} is {accuracy:.1f} %')
        

    def set_params(self, attr, value): #sets parameters
        setattr(self, attr, value)
        
    def props(cls):   
        return [i for i in cls.__dict__.items() if i[:1] != '_']

    def get_classifier(self):
        return self.classifier()

    def set_optimiser(self, optimiser_choice):
        return 
    


In [ ]:

class MLPClassifer(nn.Module):
      '''
        Multilayer Perceptron.
      '''
      def __init__(self ):
        super().__init__()
        input=trainset.data[1].shape
        output=len(torch.unique(torch.tensor(trainset.targets)))
        hidden_layer_sizes=(64,32)
        self.layers = nn.Sequential(
          nn.Flatten(),
          nn.Linear(reduce(operator.mul, input),  hidden_layer_sizes[0]),
          nn.ReLU(),
          nn.Linear(hidden_layer_sizes[0],  hidden_layer_sizes[1]),
          nn.ReLU(),
          nn.Linear(hidden_layer_sizes[1], output)
        )

      def forward(self, x):
        '''Forward pass'''
        return self.layers(x)

class Neural_Network(object):
    
    
    def __init__ (self,batch_size=2000,epochs=2,num_classes=0,verbose=True,running_loss=0,loss_function = nn.CrossEntropyLoss(),
         optimiser_choice='adam',classifier= MLPClassifer(), 
         learning_rate=1e-3,  betas=(0.9,0.99) , eps=1e-8, weight_decay=0, amsgrad=False,
      momentum=0.9, nesterov=True, dampening=0):
      
      self.batch_size=batch_size
      self.epochs=epochs
      #used for label count 
      self.num_classes=0
      self.verbose=True
        
      self.running_loss = running_loss
      self.loss_function = loss_function
       
      self.classifier= classifier
      self.optimiser_choice=optimiser_choice
      
      #needed for extraxting the probabilities 
      self.sm = torch.nn.Softmax()

      #some initial parameters for either standard adam ot sgd

      self.learning_rate=learning_rate 
      self.betas= betas
      self.eps= eps
      self.weight_decay=weight_decay
      self.amsgrad=amsgrad 
      self.momentum=momentum
      self.nesterov=nesterov
      self.dampening=dampening
    
      self.set_optimiser(self.optimiser_choice)
      
      self.optimiser=None 
      if  optimiser_choice.lower().strip()=='adam':
          self.optimiser=torch.optim.Adam(self.classifier.parameters(),lr=self.learning_rate, betas=self.betas, eps=self.eps, weight_decay=self.weight_decay, amsgrad=self.amsgrad)
      elif optimiser_choice.lower().strip()=='sgd':
          self.optimiser=torch.optim.SGD(self.classifier.parameters(),lr=self.learning_rate, weight_decay=self.weight_decay, momentum=0.9,nesterov=self.nesterov , dampening=self.dampening)
      

    def get_optimiser(self):
      print(self.optimiser)


    def set_optimiser(self, optimiser_choice):
      #playing around with adam and sgd only at the moment
        print('We are outside this function')
        if  optimiser_choice.lower().strip()=='adam':
          self.optimiser=torch.optim.Adam(self.classifier.parameters(),lr=self.learning_rate, betas=self.betas, eps=self.eps, weight_decay=self.weight_decay, amsgrad=self.amsgrad)
          print(' Check whether the optimiser is initialised correctly')
        elif optimiser_choice.lower().strip()=='sgd':
          self.optimiser=torch.optim.SGD(self.classifier.parameters(),lr=self.learning_rate, weight_decay=self.weight_decay, momentum=0.9,nesterov=self.nesterov , dampening=self.dampening)
        return self.optimiser


    def get_params(self): #get parameters
       if (self.optimiser != None) :
         return  (self.props(),self.optimiser.state_dict(), self.classifier.state_dict)
       else: return self.classifier.state_dict() 
  
    def fit(self,trainloader):
        for epoch in range(self.epochs):  # loop over the dataset 2 times

          running_loss =self.running_loss
          for i, data in enumerate(trainloader, 0):
          # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
             
            # zero the parameter gradients
            self.optimiser.zero_grad()

            # forward + compute loss+ backward + optimize
            outputs = self.classifier(inputs)
            loss = self.loss_function(outputs, labels)
            loss.backward()
            self.optimiser.step()
            # print statistics
            running_loss += loss.item()
            if i % self.batch_size == 1999:    # print every 2000 mini-batches
                if self.verbose==True:
                  print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / self.batch_size:.3f}')
                running_loss = 0.0

    def predict(self,testloader): #Predict using the multi-layer perceptron classifier.
      prediction_list=[]
      # again no gradients needed
      with torch.no_grad():
          for data in testloader:
              images, labels = data
              outputs = self.classifier(images)
              estimation, predictions = torch.max(outputs, 1)
              prediction_list.append(predictions)
      return prediction_list #torch.stack(prediction_list)


    def predict_log_proba(self,loader):  #	Return the log of probability estimates.
        y_prob=self.predict_proba(loader)
        log_proba=np.log(y_prob)
        return log_proba
 

    def predict_proba(self,loader):	#Probability estimates.
      probabilities_list=[]
      # again no gradients needed
      with torch.no_grad():
          for data in loader:
              images, labels = data
              outputs = self.classifier(images)
              probabilities_list.append(self.sm(outputs) )
              estimation, predictions = torch.max(outputs, 1)
      return probabilities_list
      

    def score(self,loader): #Return the mean accuracy on the given test data and labels.
        prediction_list=[]
        if self.num_classes==0:
            classes = ('plane', 'car', 'bird', 'cat',
                    'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
            # prepare to count predictions for each class
            correct_pred = {classname: 0 for classname in classes}
            total_pred = {classname: 0 for classname in classes}
        #use the class number as key
        else: 
            classes=tuple(np.arange(self.num_classes))  
            correct_pred = {classname: 0 for classname in np.arange(self.num_classes)}
            total_pred= {classname: 0 for classname in np.arange(self.num_classes)}

        # again no gradients needed
        with torch.no_grad():
            for data in loader:
                images, labels = data
                outputs = self.classifier(images)
                estimation, predictions = torch.max(outputs, 1)
                prediction_list.append(predictions)
                # collect the correct predictions for each class
                for label, prediction in zip(labels, predictions):
                    if label == prediction:
                        correct_pred[classes[label]] += 1
                    total_pred[classes[label]] += 1
        accuracies={}
        # print accuracy for each class
        for classname, correct_count in correct_pred.items():
            accuracy = 100 * float(correct_count) / total_pred[classname]
            accuracies[classname]=accuracy
            if self.verbose:
              print(f'Accuracy for class: {classname} is {accuracy:.1f} %')
        

    def set_params(self, attr, value): #sets parameters
        setattr(self, attr, value)
        
    def props(cls):   
        return [i for i in cls.__dict__.items() if i[:1] != '_']

    def get_classifier(self):
        return self.classifier()

    def set_optimiser(self, optimiser_choice):
        return 
    


In [ ]:
model= Neural_Network()
model.classifier= MLPClassifer()
model.set_params('optimiser_choice','adam')
model.get_optimiser()

 Check whether the optimiser is initialised correctly
Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.99)
    capturable: False
    eps: 1e-08
    foreach: None
    lr: 0.001
    maximize: False
    weight_decay: 0
)


In [ ]:
model= Neural_Network()
model.classifier= MLPClassifer()
model.set_params('optimiser_choice','adam')
model.get_optimiser()
#model.fit(trainloader)#
#print( list(MLP().named_parameters()))


None


In [ ]:
model=Trainer()
model.set_optimiser('adam')
model.set_params('verbose', False)
model.get_params()
model.fit(trainloader)
model.predict(testloader)
model.score(testloader)
model.predict_log_proba(testloader)
model.predict_log_proba(testloader)

KeyboardInterrupt: ignored